<a href="https://colab.research.google.com/github/Wangila/GenAiBootCamp/blob/master/Day_1_Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Day 1 - Prompting

Welcome to the Kaggle 5-day Generative AI course!

This notebook will show you how to get started with the Gemini API and walk you through some of the example prompts and techniques that you can also read about in the Prompting whitepaper. You don't need to read the whitepaper to use this notebook, but the papers will give you some theoretical context and background to complement this interactive notebook.


## Before you begin

In this notebook, you'll start exploring prompting using the Python SDK and AI Studio. For some inspiration, you might enjoy exploring some apps that have been built using the Gemini family of models. Here are a few that we like, and we think you will too.

* [TextFX](https://textfx.withgoogle.com/) is a suite of AI-powered tools for rappers, made in collaboration with Lupe Fiasco,
* [SQL Talk](https://sql-talk-r5gdynozbq-uc.a.run.app/) shows how you can talk directly to a database using the Gemini API,
* [NotebookLM](https://notebooklm.google/) uses Gemini models to build your own personal AI research assistant.


## For help

**Common issues are covered in the [FAQ and troubleshooting guide](https://www.kaggle.com/code/markishere/day-0-troubleshooting-and-faqs).**

## New for Gemini 2.0!

This course material was first launched in November 2024. The AI and LLM space is moving incredibly fast, so we have made some updates to use the latest models and capabilities.

* These codelabs have been updated to use the Gemini 2.0 family of models.
* The Python SDK has been updated from `google-generativeai` to the new, unified [`google-genai`](https://pypi.org/project/google-genai) SDK.
  * This new SDK works with both the developer Gemini API as well as Google Cloud Vertex AI, and switching is [as simple as changing some fields](https://pypi.org/project/google-genai/#:~:text=.Client%28%29-,API%20Selection,-By%20default%2C%20the).
* New model capabilities have been added to the relevant codelabs, such as "thinking mode" in this lab.
* Day 1 includes a new [Evaluation codelab](https://www.kaggle.com/code/markishere/day-1-evaluation-and-structured-output).

## Get started with Kaggle notebooks

If this is your first time using a Kaggle notebook, welcome! You can read about how to use Kaggle notebooks [in the docs](https://www.kaggle.com/docs/notebooks).

First, you will need to phone verify your account at kaggle.com/settings.

![](https://storage.googleapis.com/kaggle-media/Images/5dgai_0.png)

To run this notebook, as well as the others in this course, you will need to make a copy, or fork, the notebook. Look for the `Copy and Edit` button in the top-right, and **click it** to make an editable, private copy of the notebook. It should look like this one:

![Copy and Edit button](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_1.png)

Your copy will now have a ▶️ **Run** button next to each code cell that you can press to execute that cell. These notebooks are expected to be run in order from top-to-bottom, but you are encouraged to add new cells, run your own code and explore. If you get stuck, you can try the `Factory reset` option in the `Run` menu, or head back to the original notebook and make a fresh copy.

![Run cell button](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_2.png)

### Problems?

If you have any problems, head over to the [Kaggle Discord](https://discord.com/invite/kaggle), find the [`#5dgai-q-and-a` channel](https://discord.com/channels/1101210829807956100/1303438695143178251) and ask for help.

## Get started with the Gemini API

All of the exercises in this notebook will use the [Gemini API](https://ai.google.dev/gemini-api/) by way of the [Python SDK](https://pypi.org/project/google-genai/). Each of these prompts can be accessed directly in [Google AI Studio](https://aistudio.google.com/) too, so if you would rather use a web interface and skip the code for this activity, look for the <img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> AI Studio link on each prompt.

Next, you will need to add your API key to your Kaggle Notebook as a Kaggle User Secret.

![](https://storage.googleapis.com/kaggle-media/Images/5dgai_1.png)
![](https://storage.googleapis.com/kaggle-media/Images/5dgai_2.png)
![](https://storage.googleapis.com/kaggle-media/Images/5dgai_3.png)
![](https://storage.googleapis.com/kaggle-media/Images/5dgai_4.png)

### Install the SDK

In [1]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.5 MB/s eta 0:00:00


Import the SDK and some helpers for rendering the output.

In [2]:
#Google GenAI
from google import genai
from google.genai import types

#OpenAI
from openai import OpenAI



from IPython.display import HTML, Markdown, display

Set up a retry helper. This allows you to "Run all" without worrying about per-minute quota.

In [3]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [4]:
#from kaggle_secrets import UserSecretsClient

#GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

If you received an error response along the lines of `No user secrets exist for kernel id ...`, then you need to add your API key via `Add-ons`, `Secrets` **and** enable it.

![Screenshot of the checkbox to enable GOOGLE_API_KEY secret](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

### Run your first prompt

In this step, you will test that your API key is set up correctly by making a request.

The Python SDK uses a [`Client` object](https://googleapis.github.io/python-genai/genai.html#genai.client.Client) to make requests to the API. The client lets you control which back-end to use (between the Gemini API and Vertex AI) and handles authentication (the API key).

The `gemini-2.0-flash` model has been selected here.

**Note**: If you see a `TransportError` on this step, you may need to **🔁 Factory reset** the notebook one time.

In [5]:
client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Explain AI to me like I'm a kid.")

print(response.text)

Imagine you have a really smart puppy. You teach it tricks by showing it lots of examples. You show it pictures of cats and tell it "This is a cat!" Over and over and over.

Eventually, the puppy can see a new picture of a cat and say, "That's a cat!" even though it's never seen *that* exact cat before.

That's kind of what AI is! It's like a super smart puppy, but instead of learning about cats, it can learn about almost anything!

We give computers tons of examples and tell them what to look for.  They learn patterns and then can use those patterns to do things like:

*   **Recognize your face on your phone:** You show it lots of pictures of you, and it learns what *you* look like!
*   **Suggest videos you might like:** It watches what you like to watch and figures out what other videos are similar.
*   **Help your parents drive their car:** It can see things like traffic lights and other cars and help them drive safely!

So, AI is just a fancy way of teaching computers to learn thin

The response often comes back in markdown format, which you can render directly in this notebook.

In [6]:
Markdown(response.text)

Imagine you have a really smart puppy. You teach it tricks by showing it lots of examples. You show it pictures of cats and tell it "This is a cat!" Over and over and over.

Eventually, the puppy can see a new picture of a cat and say, "That's a cat!" even though it's never seen *that* exact cat before.

That's kind of what AI is! It's like a super smart puppy, but instead of learning about cats, it can learn about almost anything!

We give computers tons of examples and tell them what to look for.  They learn patterns and then can use those patterns to do things like:

*   **Recognize your face on your phone:** You show it lots of pictures of you, and it learns what *you* look like!
*   **Suggest videos you might like:** It watches what you like to watch and figures out what other videos are similar.
*   **Help your parents drive their car:** It can see things like traffic lights and other cars and help them drive safely!

So, AI is just a fancy way of teaching computers to learn things and do clever stuff, just like a really smart puppy!


### Start a chat

The previous example uses a single-turn, text-in/text-out structure, but you can also set up a multi-turn chat structure too.

In [7]:
chat = client.chats.create(model='gemini-2.0-flash', history=[])
response = chat.send_message('Hello! My name is Zlork.')
print(response.text)

Hello Zlork! It's nice to meet you. How can I help you today?



In [8]:
response = chat.send_message('Can you tell me something interesting about dinosaurs?')
print(response.text)

Okay, here's a fascinating dinosaur fact for you:

**Did you know that some dinosaurs had feathers?**

While we often picture dinosaurs as scaly reptiles, evidence suggests that many theropods (the group that includes T. Rex and velociraptors) actually had feathers. These feathers weren't necessarily for flight; they could have been used for insulation, display, or even camouflage. Discoveries in China, in particular, have revealed incredibly well-preserved feathered dinosaur fossils. It's changing the way we think about what dinosaurs looked like!

Pretty cool, right?



While you have the `chat` object alive, the conversation state
persists. Confirm that by asking if it knows the user's name.

In [9]:
response = chat.send_message('Do you remember what my name is?')
print(response.text)

Yes, Zlork. I remember your name.



**OpenAI** - test how same prompts do with OpenAI

In [10]:
openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [11]:
openai_response = openai_client.responses.create(
    model='gpt-4.1',
    input ="Explain AI to me like I'm a kid."
)

print(openai_response.output_text)

Okay! Imagine you have a really smart robot friend. This friend is called **AI**, which stands for **Artificial Intelligence**. 

AI can't play outside or eat ice cream, but it can **learn things** and **help you with tasks**. For example, it can answer questions, help you find your favorite cartoon, or even draw pictures for you.

It learns by looking at lots and lots of examples. Just like how you learn how to draw by copying pictures or how you learn words by listening to people talk, AI learns from information we give it.

So, AI is like a super helpful clever robot brain that can help you with many things—just like a friendly assistant!


In [12]:
Markdown(openai_response.output_text)

Okay! Imagine you have a really smart robot friend. This friend is called **AI**, which stands for **Artificial Intelligence**. 

AI can't play outside or eat ice cream, but it can **learn things** and **help you with tasks**. For example, it can answer questions, help you find your favorite cartoon, or even draw pictures for you.

It learns by looking at lots and lots of examples. Just like how you learn how to draw by copying pictures or how you learn words by listening to people talk, AI learns from information we give it.

So, AI is like a super helpful clever robot brain that can help you with many things—just like a friendly assistant!

Start a Chat in OpenAI

**Manually manage conversation state**

In [13]:
openai_chat_history = [
    {
        "role": "user",
        "content": "tell me a joke"
    }
]

response = openai_client.responses.create(
    model="gpt-4o-mini",
    input=openai_chat_history,
    store = False
)

print(response.output_text)

Why don't skeletons fight each other?

They don't have the guts!


In [14]:
# Add the response to the conversation
openai_chat_history += [{"role": el.role, "content": el.content} for el in response.output]

openai_chat_history.append({ "role": "user", "content": "tell me another" })

second_response = openai_client.responses.create(
    model="gpt-4o-mini",
    input=openai_chat_history,
    store=False
)

print(second_response.output_text)

Why did the scarecrow win an award?

Because he was outstanding in his field!


In [15]:
chat_response = openai_client.responses.create(
    model='gpt-4.1',
    input = 'Hello! My name is Zlork.'
  )

print(chat_response.output_text)

Hello, Zlork! It's great to meet you. How can I assist you today? 😊


In [16]:
chat_response = openai_client.responses.create(
   model='gpt-4.1',
   previous_response_id=chat_response.id,
    input = 'Can you tell me something interesting about dinosaurs?'
  )
print(chat_response.output_text)

Absolutely, Zlork! Dinosaurs are fascinating creatures with many interesting facts. Here’s one you might enjoy:

**Some dinosaurs had feathers!**  
While we often picture dinosaurs as giant, scaly reptiles, many species, especially among the theropod group (which includes **Velociraptor** and even **Tyrannosaurus rex** in some stages of their lives), actually had feathers. These feathers might have been used for insulation, display, or even gliding and flight in some smaller species. In fact, modern birds are considered living dinosaurs—they evolved from small, feathered theropods!

If you want to know more cool facts about a specific dinosaur, just let me know which one!


In [17]:
chat_response = openai_client.responses.create(
   model='gpt-4.1',
   previous_response_id=chat_response.id,
    input = 'Do you remember what my name is?'
  )
print(chat_response.output_text)

Yes, your name is Zlork! If there’s anything else you’d like to talk about or if you have more questions—dinosaur-related or not—just let me know! 😊


**OpenAI APIs for conversation state**

In [18]:
from openai import OpenAI


response = openai_client.responses.create(
    model="gpt-4o-mini",
    input="tell me a joke",
)
print(response.output_text)

second_response = openai_client.responses.create(
    model="gpt-4o-mini",
    previous_response_id=response.id,
    input=[{"role": "user", "content": "explain why this is funny."}],
)
print(second_response.output_text)

Why don't scientists trust atoms?

Because they make up everything!
The joke plays on a double meaning. 

1. **Literal Meaning**: Atoms are the basic building blocks of matter, meaning everything in the physical world is made up of atoms.
  
2. **Figurative Meaning**: The phrase "make up" can also mean to invent or to fabricate stories, suggesting untrustworthiness.

So, the humor comes from the clever wordplay that connects these two meanings, creating an unexpected twist that’s both educational and amusing.


### Choose a model

The Gemini API provides access to a number of models from the Gemini model family. Read about the available models and their capabilities on the [model overview page](https://ai.google.dev/gemini-api/docs/models/gemini).

In this step you'll use the API to list all of the available models.

In [19]:
for model in client.models.list():
  print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-p

The [`models.list`](https://ai.google.dev/api/models#method:-models.list) response also returns additional information about the model's capabilities, like the token limits and supported parameters.

In [20]:
from pprint import pprint

for model in client.models.list():
  if model.name == 'models/gemini-2.0-flash':
    pprint(model.to_json_dict())
    break

{'description': 'Gemini 2.0 Flash',
 'display_name': 'Gemini 2.0 Flash',
 'input_token_limit': 1048576,
 'name': 'models/gemini-2.0-flash',
 'output_token_limit': 8192,
 'supported_actions': ['generateContent', 'countTokens', 'createCachedContent'],
 'tuned_model_info': {},
 'version': '2.0'}


In [21]:
#List OpenAI models
for model in openai_client.models.list():
  print(model.id)

gpt-4o-audio-preview-2024-12-17
dall-e-3
dall-e-2
gpt-4o-audio-preview-2024-10-01
gpt-4-turbo-preview
text-embedding-3-small
gpt-4.1-nano
gpt-4.1-nano-2025-04-14
gpt-4o-realtime-preview-2024-10-01
gpt-4o-realtime-preview
babbage-002
o1-mini-2024-09-12
o1-mini
gpt-4
text-embedding-ada-002
chatgpt-4o-latest
gpt-4o-mini-audio-preview
gpt-4o-audio-preview
o1-preview-2024-09-12
gpt-4o-mini-realtime-preview
gpt-4.1-mini
gpt-4o-mini-realtime-preview-2024-12-17
gpt-3.5-turbo-instruct-0914
gpt-4o-mini-search-preview
gpt-4.1-mini-2025-04-14
o1
gpt-3.5-turbo-16k
o1-2024-12-17
davinci-002
gpt-3.5-turbo-1106
gpt-4o-search-preview
gpt-3.5-turbo-instruct
gpt-3.5-turbo
gpt-4o-mini-search-preview-2025-03-11
gpt-4-0125-preview
gpt-4o-2024-11-20
whisper-1
gpt-4o-2024-05-13
o1-pro
o1-pro-2025-03-19
o1-preview
gpt-4-1106-preview
gpt-4-0613
text-embedding-3-large
gpt-4o-mini-tts
gpt-4o-transcribe
gpt-4.5-preview
gpt-4.5-preview-2025-02-27
gpt-4o-search-preview-2025-03-11
omni-moderation-2024-09-26
o3-mini
o

## Explore generation parameters



### Output length

When generating text with an LLM, the output length affects cost and performance. Generating more tokens increases computation, leading to higher energy consumption, latency, and cost.

To stop the model from generating tokens past a limit, you can specify the `max_output_tokens` parameter when using the Gemini API. Specifying this parameter does not influence the generation of the output tokens, so the output will not become more stylistically or textually succinct, but it will stop generating tokens once the specified length is reached. Prompt engineering may be required to generate a more complete output for your given limit.

In [22]:
from google.genai import types

short_config = types.GenerateContentConfig(max_output_tokens=200)

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents='Write a 1000 word essay on the importance of olives in modern society.')

print(response.text)

## The Enduring Olive: A Cornerstone of Culture, Cuisine, and Commerce in Modern Society

The olive, a humble drupe cultivated for millennia, continues to be a remarkably important element of modern society. More than just a tasty snack or a source of healthy oil, the olive embodies a rich tapestry of cultural significance, culinary versatility, and economic impact, extending its influence across continents and impacting diverse aspects of contemporary life. From the sun-drenched groves of the Mediterranean to the bustling grocery stores of the West, the olive remains a powerful and enduring symbol of prosperity, peace, and palatable pleasure.

Historically, the olive tree (Olea europaea) has been deeply intertwined with the development of civilization, particularly in the Mediterranean region. Its cultivation dates back to the Bronze Age, and its presence permeates the mythology, art, and religious practices of ancient Greece and Rome. The olive branch, a symbol of peace and victory, 

In [23]:
#Controlling output length OpenAI
response = openai_client.responses.create(
    model='gpt-4.1',
    max_output_tokens=100,
    input = 'Write a 1000 word essay on the importance of olives in modern society.'
    )

print(response.output_text)

**The Importance of Olives in Modern Society**

Olives, scientifically known as *Olea europaea*, are among the oldest cultivated trees in the world, with a history stretching back over 6,000 years. Native to the Mediterranean basin, olives and their by-products, particularly olive oil, have woven themselves into the fabric of civilizations. Today, in the 21st century, olives hold profound significance, touching diverse sectors including nutrition, health, economics, culture, and the environment. The


In [24]:
Markdown(response.output_text)

**The Importance of Olives in Modern Society**

Olives, scientifically known as *Olea europaea*, are among the oldest cultivated trees in the world, with a history stretching back over 6,000 years. Native to the Mediterranean basin, olives and their by-products, particularly olive oil, have woven themselves into the fabric of civilizations. Today, in the 21st century, olives hold profound significance, touching diverse sectors including nutrition, health, economics, culture, and the environment. The

In [25]:
response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents='Write a short poem on the importance of olives in modern society.')

print(response.text)

From ancient groves to modern plate,
An olive's journey, a twist of fate.
Pressed to gold, a liquid gleam,
Enriching flavors, a foodie dream.

In salads bright, a salty grace,
Or tapenade, a vibrant space.
A symbol of peace, a healthy bite,
The olive's presence, pure and bright.



In [26]:
#Controlling output length OpenAI
response = openai_client.responses.create(
    model='gpt-4.1',
    max_output_tokens=200,
    input = 'Write a short poem on the importance of olives in modern society.'
    )

print(response.output_text)

In groves where ancient whispers dwell,  
The olive’s fruit, a tale to tell—  
Of peace it speaks, of health it brings,  
In bowls and jars and savory things.

A drop of oil, liquid gold,  
On salads bright, or breads of old;  
It graces tables far and near,  
Uniting friends, reliving cheer.

Not just a crop, but symbol true,  
Of hope and bonds between the few  
Who gather ‘round, and through the years,  
Share laughter, stories, even tears.

Oh, humble olive, green or black—  
You bridge the past, you lead the pack;  
In every kitchen’s heart, you stay—  
A gift from time, here, every day.


Explore with your own prompts. Try a prompt with a restrictive output limit and then adjust the prompt to work within that limit.

### Temperature

Temperature controls the degree of randomness in token selection. Higher temperatures result in a higher number of candidate tokens from which the next output token is selected, and can produce more diverse results, while lower temperatures have the opposite effect, such that a temperature of 0 results in greedy decoding, selecting the most probable token at each step.

Temperature doesn't provide any guarantees of randomness, but it can be used to "nudge" the output somewhat.

In [27]:
high_temp_config = types.GenerateContentConfig(temperature=2.0)


for _ in range(5):
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=high_temp_config,
      contents='Pick a random colour... (respond in a single word)')

  if response.text:
    print(response.text, '-' * 25)

Orange
 -------------------------
Cerulean.
 -------------------------
Magenta
 -------------------------
Purple
 -------------------------
Azure
 -------------------------


In [28]:
#OpenAI temperature
for _ in range(5):
  response = openai_client.responses.create(
      model='gpt-4.1',
      temperature=2.0,
      input ='Pick a random colour... (respond in a single word)')

  if response.output_text:
    print(response.output_text, '\n','-' * 25)

Azure 
 -------------------------
Turquoise 
 -------------------------
Cerulean 
 -------------------------
Turquoise 
 -------------------------
Turquoise 
 -------------------------


Now try the same prompt with temperature set to zero. Note that the output is not completely deterministic, as other parameters affect token selection, but the results will tend to be more stable.

In [29]:
low_temp_config = types.GenerateContentConfig(temperature=0.0)

for _ in range(5):
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=low_temp_config,
      contents='Pick a random colour... (respond in a single word)')

  if response.text:
    print(response.text, '-' * 25)

Azure
 -------------------------
Azure
 -------------------------
Azure
 -------------------------
Azure
 -------------------------
Azure
 -------------------------


In [30]:
#OpenAI temperature
for _ in range(5):
  response = openai_client.responses.create(
      model='gpt-4.1',
      temperature=0.0,
      input ='Pick a random colour... (respond in a single word)')

  if response.output_text:
    print(response.output_text, '\n','-' * 25)

Turquoise 
 -------------------------
Turquoise 
 -------------------------
Turquoise 
 -------------------------
Turquoise 
 -------------------------
Turquoise 
 -------------------------


### Top-P

Like temperature, the top-P parameter is also used to control the diversity of the model's output.

Top-P defines the probability threshold that, once cumulatively exceeded, tokens stop being selected as candidates. A top-P of 0 is typically equivalent to greedy decoding, and a top-P of 1 typically selects every token in the model's vocabulary.

You may also see top-K referenced in LLM literature. Top-K is not configurable in the Gemini 2.0 series of models, but can be changed in older models. Top-K is a positive integer that defines the number of most probable tokens from which to select the output token. A top-K of 1 selects a single token, performing greedy decoding.


Run this example a number of times, change the settings and observe the change in output.

In [ ]:
model_config = types.GenerateContentConfig(
    # These are the default values for gemini-2.0-flash.
    temperature=1.0,
    top_p=0.95,
)

story_prompt = "You are a creative writer. Write a short story about a cat who goes on an adventure."
response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=model_config,
    contents=story_prompt)

print(response.text)

Clementine was, by all accounts, a creature of habit. Every morning, she'd stretch languidly, demanding breakfast precisely at seven. Every afternoon, she'd nap in the sunbeam that painted a warm rectangle on the living room rug. Every evening, she'd weave through her human, Eleanor's, legs, purring a rumbling symphony. Life in Eleanor's cottage was predictable, comfortable, and utterly, gloriously, boring.

One blustery Tuesday, Clementine decided she'd had enough. The wind howled, rattling the windows like impatient ghosts, and a thrill, a shiver of rebellion, shot through her ginger fur. She watched a rogue leaf, a vibrant crimson maple, pirouette past the window, beckoning. That was it. Adventure called.

Slinking to the back door, Clementine waited for Eleanor to open it. A strategic leg-rub, a calculatedly innocent meow, and Eleanor, oblivious to the simmering rebellion, obliged. "Just for a minute, Clem," she warned, bending to scratch behind Clementine's ears.

Clementine waste

In [ ]:
#OpenAI temperature

response = openai_client.responses.create(
      model='gpt-4.1',
      temperature=1.0,
      top_p=0.95,
      input ='You are a creative writer. Write a short story about a cat who goes on an adventure.')

print(response.output_text)



In the sun-dappled alley behind the blue house on Maple Street lived a sleek black cat named Whiskers. He was a curious sort, with eyes like polished copper coins and a tail that twitched at the faintest flutter of a moth. Whiskers had explored every rooftop, every nook and cranny in the neighborhood—at least, he thought he had.

One bright morning, as the robins chattered and the dew still glistened on the grass, Whiskers spotted something unusual near the garden fence: a tiny, golden key lying beside a dandelion. No human scent lingered on it, only a faint aroma of wildflowers and something ancient, like old parchment.

Whiskers batted the key curiously, and as he did, a strange breeze ruffled his fur. With a flick of his tail, he followed the scent, squeezing through a gap he’d never noticed before. On the other side, the alley melted away, and Whiskers found himself in a sunlit meadow where the grass hummed with the sound of a thousand hidden insects.

There, in the very center, st

## Prompting

This section contains some prompts from the chapter for you to try out directly in the API. Try changing the text here to see how each prompt performs with different instructions, more examples, or any other changes you can think of.

### Zero-shot

Zero-shot prompts are prompts that describe the request for the model directly.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1gzKKgDHwkAvexG5Up0LMtl1-6jKMKe4g"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [ ]:
model_config = types.GenerateContentConfig(
    temperature=0.1,
    top_p=1,
    max_output_tokens=5,
)

zero_shot_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. I wish there were more movies like this masterpiece.
Sentiment: """

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=model_config,
    contents=zero_shot_prompt)

print(response.text)

POSITIVE



In [ ]:
#OpenAI zero_shot_prompt
zero_shot_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. I wish there were more movies like this masterpiece.
Sentiment: """

response = openai_client.responses.create(
    model='gpt-4.1',
    temperature = 0.1,
    top_p = 1,
    input=zero_shot_prompt)

print(response.output_text)

Sentiment: POSITIVE


#### Enum mode

The models are trained to generate text, and while the Gemini 2.0 models are great at following instructions, other models can sometimes produce more text than you may wish for. In the preceding example, the model will output the label, but sometimes it can include a preceding "Sentiment" label, and without an output token limit, it may also add explanatory text afterwards. See [this prompt in AI Studio](https://aistudio.google.com/prompts/1gzKKgDHwkAvexG5Up0LMtl1-6jKMKe4g) for an example.

The Gemini API has an [Enum mode](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Enum.ipynb) feature that allows you to constrain the output to a fixed set of values.

In [ ]:
import enum

class Sentiment(enum.Enum):
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    NEGATIVE = "negative"


response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        response_mime_type="text/x.enum",
        response_schema=Sentiment
    ),
    contents=zero_shot_prompt
    )

print(response.text)

positive


Sentiment: POSITIVE

Explanation: Despite describing the film as "disturbing," the reviewer calls it a "masterpiece" and expresses a wish for more movies like it, indicating strong approval.


When using constrained output like an enum, the Python SDK will attempt to convert the model's text response into a Python object automatically. It's stored in the `response.parsed` field.

In [ ]:
enum_response = response.parsed
print(enum_response)
print(type(enum_response))

Sentiment.POSITIVE
<enum 'Sentiment'>


### One-shot and few-shot

Providing an example of the expected response is known as a "one-shot" prompt. When you provide multiple examples, it is a "few-shot" prompt.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1jjWkjUSoMXmLvMJ7IzADr_GxHPJVV2bg"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>


In [31]:
few_shot_prompt = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "pepperoni"]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}
```

ORDER:
"""

customer_order = "Give me a large with cheese & pineapple"

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ),
    contents=[few_shot_prompt, customer_order])

print(response.text)

```json
{
"size": "large",
"type": "normal",
"ingredients": ["cheese", "pineapple"]
}
```



In [32]:

openai_few_shot_prompt = """
# Identity

You're a server at a resturant that helps people make order.

# Instructions

Parse a customer's pizza order into valid JSON:

# Examples

<product_review id="example-1">
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "pepperoni"]
}
```
</product_review>

<product_review id="example-2">
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}
```
</product_review>

ORDER:
"""


response = openai_client.responses.create(
    model='gpt-4.1',
    temperature=0.1,
    top_p=1,
    max_output_tokens = 250,
    instructions = openai_few_shot_prompt,
    input="Give me a large with cheese & pineapple"
    )

print(response.output_text)

```
{
"size": "large",
"type": "normal",
"ingredients": ["cheese", "pineapple"]
}
```


#### JSON mode

To provide control over the schema, and to ensure that you only receive JSON (with no other text or markdown), you can use the Gemini API's [JSON mode](https://github.com/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb). This forces the model to constrain decoding, such that token selection is guided by the supplied schema.

In [33]:
import typing_extensions as typing

class PizzaOrder(typing.TypedDict):
    size: str
    ingredients: list[str]
    type: str


response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=PizzaOrder,
    ),
    contents="Can I have a large dessert pizza with apple and chocolate")

print(response.text)

{
  "size": "large",
  "ingredients": ["apple", "chocolate"],
  "type": "dessert"
}


In [34]:
#OpenAI JSON mode
from pydantic import BaseModel
class PizzaOrder(BaseModel):
    size: str
    ingredients: list[str]
    type: str

response = openai_client.responses.parse(
    model='gpt-4.1',
    temperature=0.1,
    instructions = openai_few_shot_prompt,
    input="Give me a large with cheese & pineapple",
    text_format=PizzaOrder
)

print(response.output_text)

{
  "size": "large",
  "ingredients": ["cheese", "pineapple"],
  "type": "normal"
}


In [35]:
class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

response = openai_client.responses.parse(
    model="gpt-4.1",
    temperature=0.1,
    input=[
        {
            "role": "system",
            "content": "You are a helpful math tutor. Guide the user through the solution step by step.",
        },
        {"role": "user", "content": "how can I solve 8x + 7 = -23"},
    ],
    text_format=MathReasoning,
)

print(response.output_text)

{"steps":[{"explanation":"Start with the equation: 8x + 7 = -23. The goal is to solve for x. First, subtract 7 from both sides to isolate the term with x.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"Simplify both sides. 7 - 7 is 0, so you have 8x = -30.","output":"8x = -30"},{"explanation":"Now, divide both sides by 8 to solve for x.","output":"8x / 8 = -30 / 8"},{"explanation":"Simplify the right side. -30 divided by 8 is -3.75.","output":"x = -3.75"}],"final_answer":"x = -3.75"}


### Chain of Thought (CoT)

Direct prompting on LLMs can return answers quickly and (in terms of output token usage) efficiently, but they can be prone to hallucination. The answer may "look" correct (in terms of language and syntax) but is incorrect in terms of factuality and reasoning.

Chain-of-Thought prompting is a technique where you instruct the model to output intermediate reasoning steps, and it typically gets better results, especially when combined with few-shot examples. It is worth noting that this technique doesn't completely eliminate hallucinations, and that it tends to cost more to run, due to the increased token count.

Models like the Gemini family are trained to be "chatty" or "thoughtful" and will provide reasoning steps without prompting, so for this simple example you can ask the model to be more direct in the prompt to force a non-reasoning response. Try re-running this step if the model gets lucky and gets the answer correct on the first try.

In [36]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
am 20 years old. How old is my partner? Return the answer directly."""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt)

print(response.text)

48



In [39]:
#GenAI CoT
response = openai_client.responses.create(
    model='gpt-4.1',
    input = prompt
)

print(response.output_text)


Let's solve the problem step by step:

Let:
- Your age when you were young = 4 years
- Your partner's age at that time = 3 times your age = \(3 \times 4 = 12\) years

Now:
- Your current age = 20 years

**Step 1: Find the number of years that have passed since then.**

Years passed = Your current age – Your age then  
Years passed = \(20 - 4 = 16\) years

**Step 2: Find your partner's current age.**

Partner's age then = 12 years  
Partner's current age = Partner's age then + Years passed  
Partner's current age = \(12 + 16 = 28\) years

---

**Final Answer:**

**Your partner is 28 years old now.**


Now try the same approach, but indicate to the model that it should "think step by step".

In [37]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now,
I am 20 years old. How old is my partner? Let's think step by step."""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt)

Markdown(response.text)

Here's how to solve the problem step-by-step:

1. **Find the age difference:** When you were 4, your partner was 3 times your age, so they were 4 * 3 = 12 years old.

2. **Calculate the age difference:** The age difference between you and your partner is 12 - 4 = 8 years.

3. **Determine the partner's current age:** Since the age difference remains the same, your partner is currently 20 + 8 = 28 years old.

**Answer:** Your partner is currently 28 years old.


In [41]:
#GenAI CoT requested
response = openai_client.responses.create(
    model='gpt-4.1',
    input = prompt
)

Markdown(response.output_text)

Let’s break it down step by step:

### Step 1: Set up the variables
- Let:
  - Your age when you were 4 = 4
  - Your partner’s age at that time = \( x \)

### Step 2: Translate the relationship
- The problem says: "my partner was 3 times my age" when you were 4.
- That means when you were 4, your partner was \( 3 \times 4 = 12 \) years old.

### Step 3: Find the age difference
- When you were 4, your partner was 12.
- Age difference = \( 12 - 4 = 8 \) years

### Step 4: Fast forward to now
- Now, you are 20.
- The age difference remains the same (8 years).

### Step 5: Calculate your partner’s current age
- Partner’s current age = Your current age + age difference
- Partner’s current age = \( 20 + 8 = 28 \)

---

## **Final Answer**
**Your partner is currently 28 years old.**

### ReAct: Reason and act

In this example you will run a ReAct prompt directly in the Gemini API and perform the searching steps yourself. As this prompt follows a well-defined structure, there are frameworks available that wrap the prompt into easier-to-use APIs that make tool calls automatically, such as the LangChain example from the "Prompting" whitepaper.

To try this out with the Wikipedia search engine, check out the [Searching Wikipedia with ReAct](https://github.com/google-gemini/cookbook/blob/main/examples/Search_Wikipedia_using_ReAct.ipynb) cookbook example.


> Note: The prompt and in-context examples used here are from [https://github.com/ysymyth/ReAct](https://github.com/ysymyth/ReAct) which is published under an [MIT license](https://opensource.org/licenses/MIT), Copyright (c) 2023 Shunyu Yao.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/18oo63Lwosd-bQ6Ay51uGogB3Wk3H8XMO"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>


In [43]:
model_instructions = """
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation,
Observation is understanding relevant information from an Action's output and Action can be one of three types:
 (1) <search>entity</search>, which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it
     will return some similar entities to search and you can try to search the information from those topics.
 (2) <lookup>keyword</lookup>, which returns the next sentence containing keyword in the current context. This only does exact matches,
     so keep your searches short.
 (3) <finish>answer</finish>, which returns the answer and finishes the task.
"""

example1 = """Question
Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?

Thought 1
The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.

Action 1
<search>Milhouse</search>

Observation 1
Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.

Thought 2
The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".

Action 2
<lookup>named after</lookup>

Observation 2
Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.

Thought 3
Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.

Action 3
<finish>Richard Nixon</finish>
"""

example2 = """Question
What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?

Thought 1
I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.

Action 1
<search>Colorado orogeny</search>

Observation 1
The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought 2
It does not mention the eastern sector. So I need to look up eastern sector.

Action 2
<lookup>eastern sector</lookup>

Observation 2
The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought 3
The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.

Action 3
<search>High Plains</search>

Observation 3
High Plains refers to one of two distinct land regions

Thought 4
I need to instead search High Plains (United States).

Action 4
<search>High Plains (United States)</search>

Observation 4
The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130m).

Thought 5
High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.

Action 5
<finish>1,800 to 7,000 ft</finish>
"""

# Come up with more examples yourself, or take a look through https://github.com/ysymyth/ReAct/

To capture a single step at a time, while ignoring any hallucinated Observation steps, you will use `stop_sequences` to end the generation process. The steps are `Thought`, `Action`, `Observation`, in that order.

In [45]:
question = """Question
Who was the youngest author listed on the transformers NLP paper?
"""

# You will perform the Action; so generate up to, but not including, the Observation.
react_config = types.GenerateContentConfig(
    stop_sequences=["\nObservation"],
    system_instruction=model_instructions + example1 + example2,
)

# Create a chat that has the model instructions and examples pre-seeded.
react_chat = client.chats.create(
    model='gemini-2.0-flash',
    config=react_config,
)

resp = react_chat.send_message(question)
print(resp.text)

Thought 1
I need to find the transformers NLP paper, then find the authors, then find the youngest author.

Action 1
<search>transformers NLP paper</search>



Now you can perform this research yourself and supply it back to the model.

In [46]:
observation = """Observation 1
[1706.03762] Attention Is All You Need
Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely.
"""
resp = react_chat.send_message(observation)
print(resp.text)

Thought 2
The authors listed are Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin. I need to find the age of each author and find the youngest. This is hard to search directly, so I will start by searching for the first author.

Action 2
<search>Ashish Vaswani</search>



This process repeats until the `<finish>` action is reached. You can continue running this yourself if you like, or try the [Wikipedia example](https://github.com/google-gemini/cookbook/blob/main/examples/Search_Wikipedia_using_ReAct.ipynb) to see a fully automated ReAct system at work.

## Thinking mode

The experiemental Gemini Flash 2.0 "Thinking" model has been trained to generate the "thinking process" the model goes through as part of its response. As a result, the Flash Thinking model is capable of stronger reasoning capabilities in its responses.

Using a "thinking mode" model can provide you with high-quality responses without needing specialised prompting like the previous approaches. One reason this technique is effective is that you induce the model to generate relevant information ("brainstorming", or "thoughts") that is then used as part of the context in which the final response is generated.

Note that when you use the API, you get the final response from the model, but the thoughts are not captured. To see the intermediate thoughts, try out [the thinking mode model in AI Studio](https://aistudio.google.com/prompts/new_chat?model=gemini-2.0-flash-thinking-exp-01-21).

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1Z991SV7lZZZqioOiqIUPv9a9ix-ws4zk"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [47]:
import io
from IPython.display import Markdown, clear_output


response = client.models.generate_content_stream(
    model='gemini-2.0-flash-thinking-exp',
    contents='Who was the youngest author listed on the transformers NLP paper?',
)

buf = io.StringIO()
for chunk in response:
    buf.write(chunk.text)
    # Display the response as it is streamed
    print(chunk.text, end='')

# And then render the finished response as formatted markdown.
clear_output()
Markdown(buf.getvalue())

Okay, the paper you're referring to is "Attention Is All You Need," which introduced the Transformer architecture.

The authors listed on the paper are:
1.  Ashish Vaswani
2.  Noam Shazeer
3.  Niki Parmar
4.  Jakob Uszkoreit
5.  Llion Jones
6.  **Aidan N. Gomez**
7.  Łukasz Kaiser
8.  Illia Polosukhin

While authors' exact birth dates are not typically public information and certainly aren't listed on academic papers, based on his status at the time of the paper's publication (he was an undergraduate student at the University of Toronto interning at Google Brain), **Aidan N. Gomez** is widely believed and reported to have been the youngest author on the paper. The other authors were more senior researchers at Google Brain.

## Code prompting

### Generating code

The Gemini family of models can be used to generate code, configuration and scripts. Generating code can be helpful when learning to code, learning a new language or for rapidly generating a first draft.

It's important to be aware that since LLMs can make mistakes, and can repeat training data, it's essential to read and test your code first, and comply with any relevant licenses.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1YX71JGtzDjXQkgdes8bP6i3oH5lCRKxv"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [48]:
# The Gemini models love to talk, so it helps to specify they stick to the code if that
# is all that you want.
code_prompt = """
Write a Python function to calculate the factorial of a number. No explanation, provide only the code.
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=1,
        top_p=1,
        max_output_tokens=1024,
    ),
    contents=code_prompt)

Markdown(response.text)

```python
def factorial(n):
  if n == 0:
    return 1
  else:
    return n * factorial(n-1)
```


In [49]:
#Code Generation - OpenAI
response = openai_client.responses.create(
    model = 'gpt-4.1',
    temperature=1,
    top_p=1,
    max_output_tokens=1024,
    input = code_prompt
)

Markdown(response.output_text)

```python
def factorial(n):
    if n == 0 or n == 1:
        return 1
    else:
        return n * factorial(n - 1)
```

### Code execution

The Gemini API can automatically run generated code too, and will return the output.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/11veFr_VYEwBWcLkhNLr-maCG0G8sS_7Z"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [50]:
from pprint import pprint

config = types.GenerateContentConfig(
    tools=[types.Tool(code_execution=types.ToolCodeExecution())],
)

code_exec_prompt = """
Generate the first 14 odd prime numbers, then calculate their sum.
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=config,
    contents=code_exec_prompt)

for part in response.candidates[0].content.parts:
  pprint(part.to_json_dict())
  print("-----")

{'text': 'Okay, I can do that. First, I will generate the first 14 odd prime '
         'numbers, and then calculate their sum.\n'
         '\n'
         'Prime numbers are numbers greater than 1 that have only two factors: '
         '1 and themselves. The first few prime numbers are 2, 3, 5, 7, 11, '
         'and so on. Odd prime numbers exclude the number 2.\n'
         '\n'
         "Here's how I'll generate the list and calculate the sum:\n"
         '\n'
         '1.  Start with the first odd prime number, which is 3.\n'
         '2.  Iterate through odd numbers, checking each for primality.\n'
         '3.  Add the prime number to the list until the list contains 14 '
         'numbers.\n'
         '4.  Calculate the sum of the numbers in the list.\n'
         '\n'}
-----
{'executable_code': {'code': 'def is_prime(n):\n'
                             '    if n <= 1:\n'
                             '        return False\n'
                             '    for i in range(2, int(n

This response contains multiple parts, including an opening and closing text part that represent regular responses, an `executable_code` part that represents generated code and a `code_execution_result` part that represents the results from running the generated code.

You can explore them individually.

In [51]:
for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    elif part.executable_code:
        display(Markdown(f'```python\n{part.executable_code.code}\n```'))
    elif part.code_execution_result:
        if part.code_execution_result.outcome != 'OUTCOME_OK':
            display(Markdown(f'## Status {part.code_execution_result.outcome}'))

        display(Markdown(f'```\n{part.code_execution_result.output}\n```'))

Okay, I can do that. First, I will generate the first 14 odd prime numbers, and then calculate their sum.

Prime numbers are numbers greater than 1 that have only two factors: 1 and themselves. The first few prime numbers are 2, 3, 5, 7, 11, and so on. Odd prime numbers exclude the number 2.

Here's how I'll generate the list and calculate the sum:

1.  Start with the first odd prime number, which is 3.
2.  Iterate through odd numbers, checking each for primality.
3.  Add the prime number to the list until the list contains 14 numbers.
4.  Calculate the sum of the numbers in the list.



```python
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

primes = []
num = 3
while len(primes) < 14:
    if is_prime(num):
        primes.append(num)
    num += 2

print(f'{primes=}')

sum_of_primes = sum(primes)
print(f'{sum_of_primes=}')


```

```
primes=[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
sum_of_primes=326

```

The first 14 odd prime numbers are 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, and 47.

The sum of these numbers is 326.


### Explaining code

The Gemini family of models can explain code to you too. In this example, you pass a [bash script](https://github.com/magicmonty/bash-git-prompt) and ask some questions.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1N7LGzWzCYieyOf_7bAG4plrmkpDNmUyb"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [52]:
file_contents = !curl https://raw.githubusercontent.com/magicmonty/bash-git-prompt/refs/heads/master/gitprompt.sh

explain_prompt = f"""
Please explain what this file does at a very high level. What is it, and why would I use it?

```
{file_contents}
```
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=explain_prompt)

Markdown(response.text)

This file is a shell script designed to enhance your command-line prompt with Git repository status information. It's commonly known as a "Git prompt" or "bash-git-prompt".

Here's a breakdown:

*   **Purpose:** The primary goal is to display information about the current Git repository in your shell prompt. This includes things like the current branch, whether there are uncommitted changes, the status of staged files, and the divergence from the remote repository. It gives you a quick visual overview of your Git status without having to run `git status` manually.

*   **How it works:**
    *   It defines functions that fetch Git status information using `git` commands.
    *   It provides customization options through variables that you can set in your `.bashrc`, `.zshrc`, or similar shell configuration files. These variables control the colors, symbols, and the specific information displayed in the prompt.
    *   It loads a theme to style the prompt.  Themes define color schemes and symbols.  It supports custom themes.
    *   It's designed to work in both Bash and Zsh shells (with some compatibility considerations).
    *   It asynchronously fetches remote status to avoid slowing down the prompt.
    *   It integrates with virtual environments, showing the active environment in the prompt.

*   **Why use it?**

    *   **At-a-glance Git status:**  Provides a constant reminder of the state of your Git repository, helping you avoid accidentally committing changes to the wrong branch or forgetting to stage files.
    *   **Improved workflow:**  Reduces the need to constantly run `git status`, making your workflow more efficient.
    *   **Customization:** Allows you to tailor the prompt to your preferences, showing only the information you find most useful.
    *   **Aesthetics:**  Makes your command line look nicer and more informative.

In essence, this script aims to improve your command-line experience when working with Git repositories by providing a visually informative and customizable prompt. To use it, you typically `source` this script in your shell configuration file (e.g., `.bashrc` or `.zshrc`).


In [54]:
#Explaining Code : OpenAI
response = openai_client.responses.create(
    model = 'gpt-4.1',
    input = explain_prompt
)

Markdown(response.output_text)

### High-level Overview

This file is a **Bash/Zsh shell script** used for customizing and enhancing your shell prompt (the command line where you type commands) with information about your current [Git](https://git-scm.com/) repository's status.

**Most likely, this is a key logic file from the [`bash-git-prompt`](https://github.com/magicmonty/bash-git-prompt) project or a similar prompt tool for Git.**

---

### What does it do?

- **Shows Git status in your prompt:**  
  When you navigate into a directory that is a Git repository, your prompt dynamically updates to show your current branch, whether you're ahead/behind, changes staged/unstaged, untracked files, if you have stashes, etc.

- **Supports themes and customization:**  
  You can pick from built-in color themes or make your own, so your Git branch or status info appears exactly as you like.

- **Handles both Bash and Zsh:**  
  There are functions for async execution in Zsh and Bash, allowing compatibility with both.

- **Manages prompt efficiently:**  
  It only updates when necessary, resets cleanly, and can be toggled off/on.

---

### Why would I use it?

- **Better situational awareness:**  
  See at a glance in your prompt:
  - What Git branch you are on
  - If you’ve committed changes
  - If you need to pull/push
  - If you have unresolved conflicts or stashes

- **Time saving for developers:**  
  Eliminates the need to constantly type `git status` or `git branch` to check your repo.

- **Increase productivity and reduce mistakes:**  
  By always seeing your Git state, you avoid making commits to the wrong branch or forgetting to pull/push.

---

### When would I *not* use it?

- If you don’t use Git.
- If you prefer a minimalist or plain shell prompt.
- If you already use a different shell prompt enhancement that shows Git info.

---

### How would I use it?

Typically, you would:
1. **Install the script** (usually via `git clone` or your package manager).
2. **Source it in your `.bashrc` or `.zshrc`:**
   ```sh
   source /path/to/.git-prompt.sh
   ```
3. **Customize your prompt/theme** if desired.

---

### TL;DR

> This script automatically inserts information about your Git repository into your command-line prompt.  
> It’s for developers who want real-time Git status shown in their terminal prompt.

---

Let me know if you want a summary of specific functions or deeper breakdown of any section!

## Learn more

To learn more about prompting in depth:

* Check out the whitepaper issued with today's content,
* Try out the apps listed at the top of this notebook ([TextFX](https://textfx.withgoogle.com/), [SQL Talk](https://sql-talk-r5gdynozbq-uc.a.run.app/) and [NotebookLM](https://notebooklm.google/)),
* Read the [Introduction to Prompting](https://ai.google.dev/gemini-api/docs/prompting-intro) from the Gemini API docs,
* Explore the Gemini API's [prompt gallery](https://ai.google.dev/gemini-api/prompts) and try them out in AI Studio,
* Check out the Gemini API cookbook for [inspirational examples](https://github.com/google-gemini/cookbook/blob/main/examples/) and [educational quickstarts](https://github.com/google-gemini/cookbook/blob/main/quickstarts/).

Be sure to check out the codelabs on day 3 too, where you will explore some more advanced prompting with code execution.

And please share anything exciting you have tried in the Discord!

*- [Mark McD](https://linktr.ee/markmcd)*